In [10]:
%%writefile src/OrderContract2.sol
//SPDX-License-Identifier:GPL-3.0-or-later
pragma solidity ^0.8.0;

contract CustomerContract {
    struct Customer {
        uint id;
        string name;
        string phone;
        string home;
    }

    mapping(address => Customer) customers;
    mapping(uint => address) IdToAddr;
    mapping(address => uint) Mileage;
    mapping(address => uint) BalanceOf;

    function addCustomer(uint _id, string memory _name, string memory _phone, string memory _home) public {
        require(customers[tx.origin].id == 0);
        require(IdToAddr[_id] == address(0));
        customers[tx.origin] = Customer(_id, _name, _phone, _home);
        IdToAddr[_id] = tx.origin;
        BalanceOf[tx.origin] = (tx.origin).balance;
    }
    function withdrawMoney(uint _num) public payable {
        BalanceOf[tx.origin] -= _num;
    }
    function refundMoney(uint _num) public payable {
        BalanceOf[tx.origin] += _num;
    }
    function getHomeAddress(address _addr) public view returns(string memory) {
        Customer memory c = customers[_addr];
        return c.home;
    }
    function getId(address _addr) public view returns(uint) {
        Customer memory c = customers[_addr];
        return c.id;
    }
    function addMileage(uint _amount) public {
        Mileage[tx.origin] += _amount;
    }
    function getMileage() public view returns(uint) {
        return Mileage[tx.origin];
    }
    function refundMileage(uint _amount) public {
        Mileage[tx.origin] -= _amount;
    }
}

contract OrderContract {
    struct Order {
        uint id;
        string productName;
        uint amount;
        uint cost;
        uint timestamp;
        string status;
        string home;
    }

    mapping(address => Order) orders;
    mapping(uint => address) IdToAddr;

    address owner;
    CustomerContract c;
    uint OrderCount = 0;
    uint TotalCost = 0;
    uint[] OrderIds;
    constructor() {
        c = new CustomerContract();
        owner = msg.sender;
    }
    function addCustomer(uint _id, string memory _name, string memory _phone, string memory _home) public {
        c.addCustomer(_id, _name, _phone, _home);
    }
    function getHomeAddress(address _addr) public view returns(string memory) {
        return c.getHomeAddress(_addr);
    }
    function addMileage(uint _amount) public {
        c.addMileage(_amount);
    }
    function refundMileage(uint _amount) public {
        c.refundMileage(_amount);
    }
    function withdrawMoney(uint _num) public payable {
        c.withdrawMoney(_num);
    }
    function refundMoney(uint _num) public payable {
        c.refundMoney(_num);
    }
    function placeOrder(uint _id, string memory _productName, uint _amount, uint _cost) public payable{
        require(c.getId(tx.origin) != 0, "Customer does not exist");
        require(orders[tx.origin].id == 0, "Order ID already exists");
        require(msg.value == _cost);
        
        string memory _home = c.getHomeAddress(tx.origin);
        orders[tx.origin] = Order(_id, _productName, _amount, _cost, block.timestamp, "Ordered", _home);
        IdToAddr[_id] = tx.origin;

        OrderCount += 1;
        TotalCost += _cost;
        OrderIds.push(_id);
        withdrawMoney(_cost);
        addMileage(_cost/100);
        
    }
    function getStatus(uint _id) public view returns(string memory) {
        Order memory o = orders[IdToAddr[_id]];
        return o.status;
    }
    function updateStatus(uint _id, string memory _status) public {
        require(IdToAddr[_id] == tx.origin);
        orders[IdToAddr[_id]].status = _status;
    }
    function getOrderItem() public view returns(Order[] memory) {
        Order[] memory allOrders = new Order[](OrderCount);
        for(uint i=0; i<OrderCount; i++) {
            allOrders[i] = orders[IdToAddr[OrderIds[i]]];
        }
        return allOrders;
    }
    function getOrderById(uint _id) public view returns(uint, string memory, string memory, string memory) {
        Order memory o = orders[IdToAddr[_id]];
        return (o.id, o.productName, o.status, o.home);
    }
    function getNOrder() public view returns(uint) {
        return OrderCount;
    }
    function getTotalOrderAmount() public view returns(uint) {
        return TotalCost;
    }
    function queryBalance() public view returns(uint) {
        return address(this).balance;
    }
    function refund(uint _id) public {
        require(orders[IdToAddr[_id]].id != 0);
        OrderCount -= 1;
        TotalCost -= orders[IdToAddr[_id]].cost;
        orders[IdToAddr[_id]].status = "Refunded";
        refundMileage(orders[IdToAddr[_id]].cost/100);
        payable(owner).transfer(orders[IdToAddr[_id]].cost);
    }
}

Overwriting src/OrderContract2.sol


In [11]:
!solc-windows.exe --optimize --combined-json abi,bin src/OrderContract2.sol > src/OrderContract2.json

In [16]:
%%writefile src/OrderContractDeploy.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

var _abiBinJson = require('./OrderContract2.json');
var contractName = Object.keys(_abiBinJson.contracts);

_abi = _abiBinJson.contracts[contractName[1]].abi;
_bin = _abiBinJson.contracts[contractName[1]].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();

    await new web3.eth.Contract(_abi).deploy({data: "0x" + _bin}).estimateGas(function(err, gas) {
        if(!err) console.log("gas: " + gas);
    })

    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas:2000000}, function(err, transactionHash) {
            if(!err) console.log("hash: " + transactionHash);
        });
    console.log("The contract deployed to: " + deployed.options.address);
}
deploy()

Overwriting src/OrderContractDeploy.js


In [17]:
!node src/OrderContractDeploy.js

gas: 1912201
hash: 0xa8ed6098760b139f9c27f5e6e3788e26e45c7a7f2b4a3f0fd881b7f2edffa102
The contract deployed to: 0x0C30b3dc06eBf105B34bF040fd9b05d505B22238


In [18]:
%%writefile src/OrderContractUse.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

var _abiBinJson = require('./OrderContract2.json');
var contractName = Object.keys(_abiBinJson.contracts);

abi = _abiBinJson.contracts[contractName[1]].abi;
addr = "0x0C30b3dc06eBf105B34bF040fd9b05d505B22238";
order = new web3.eth.Contract(abi, addr);

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("acc1: " + accounts[1] + " bal1: " + await web3.eth.getBalance(accounts[1]));
    console.log("acc2: " + accounts[2] + " bal2: " + await web3.eth.getBalance(accounts[2]));
    console.log("acc3: " + accounts[3] + " bal3: " + await web3.eth.getBalance(accounts[3]));

    await order.methods.addCustomer(111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul").send({from: accounts[1], gas: 1622664});
    await order.methods.addCustomer(112, "lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul").send({from: accounts[2], gas: 1622664});
    await order.methods.addCustomer(113, "lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul").send({from: accounts[3], gas: 1622664});

    await order.methods.getHomeAddress(accounts[1]).call().then(console.log);
    await order.methods.getHomeAddress(accounts[2]).call().then(console.log);
    await order.methods.getHomeAddress(accounts[3]).call().then(console.log);

    await order.methods.placeOrder(555, "T-Shirt", 2, 1115).send({from: accounts[1], gas: 1622664, value:1115});
    await order.methods.placeOrder(556, "T-Shirt", 3, 1116).send({from: accounts[2], gas: 1622664, value:1116});
    await order.methods.placeOrder(557, "T-Shirt", 4, 1117).send({from: accounts[3], gas: 1622664, value:1117});

    await order.methods.getNOrder().call().then(console.log);
    await order.methods.getTotalOrderAmount().call().then(console.log);
    await order.methods.queryBalance().call().then(console.log);

    await order.methods.updateStatus(556, "On delivery").send({from: accounts[2], gas: 1622664});
    await order.methods.getOrderItem().call().then(function (arr) {
        for(i=0; i<arr.length; i++) {
            console.log(arr[i].id, arr[i].productName, arr[i].status, arr[i].home);
        }
    });

    await order.methods.refund(556).send({from: accounts[2], gas: 1622664});
    await order.methods.getNOrder().call().then(console.log);
    await order.methods.getTotalOrderAmount().call().then(console.log);
    await order.methods.queryBalance().call().then(console.log);

    await order.methods.getOrderItem().call().then(function (arr) {
        for(i=0; i<arr.length; i++) {
            console.log(arr[i].id, arr[i].productName, arr[i].status, arr[i].home);
        }
    });
}
doIt()

Overwriting src/OrderContractUse.js


In [19]:
!node src/OrderContractUse.js

acc1: 0xa9643989300947AC769B56E51E27fa8ec19fa399 bal1: 999982527284531624568
acc2: 0x8aDEAe08913b985ddec97e6a50c7e89C2492BBbC bal2: 999983792758179461341
acc3: 0x21189730f8dadC1D6453463589159D30f9715A47 bal3: 999985698290540751404
111 hongji-dong jongro-gu seoul
112 hongji-dong jongro-gu seoul
113 hongji-dong jongro-gu seoul
3
3348
3348
555 T-Shirt Ordered 111 hongji-dong jongro-gu seoul
556 T-Shirt On delivery 112 hongji-dong jongro-gu seoul
557 T-Shirt Ordered 113 hongji-dong jongro-gu seoul
2
2232
2232
555 T-Shirt Ordered 111 hongji-dong jongro-gu seoul
556 T-Shirt Refunded 112 hongji-dong jongro-gu seoul


# 가위바위보

In [1]:
%%writefile src/Rsp.sol
//SPDX-License-Identifier:GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Rsp {
    address owner;
    address A;
    address B;
    uint result;

    mapping(address => uint) AddToRsp;
    mapping(address => uint) BalanceOf;
    constructor() {
        owner = msg.sender;
    }
    function setA() public payable {
        A = tx.origin;
        AddToRsp[A] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp)))%3);
        BalanceOf[A] = (A).balance;
        require(msg.value == 1000);
        BalanceOf[A] -= 1000;
    }
    function setB(uint8 user) public payable{
        require(user >= 0 && user <= 2);
        B = tx.origin;
        AddToRsp[B] = user;
        BalanceOf[B] = (B).balance;
        require(msg.value == 1000);
        BalanceOf[B] -= 1000;
    }
    function play() public {
        if (AddToRsp[A] == AddToRsp[B]) {
            result = 0; //비겼을 때
        }
        else if ((AddToRsp[A] == 0 && AddToRsp[B] == 1) || (AddToRsp[A] == 1 && AddToRsp[B] == 2) || (AddToRsp[A] == 2 && AddToRsp[B] == 0)) {
            result = 1; // A가 이겼을 때
        }
        else {
            result = 2; // B가 이겼을 때
        }
    }
    function getMatchResult() public view returns(string memory){
        if (result == 0) {
            return "tie";
        }
        else if (result == 1) {
            return "A wins";
        }
        else {
            return "B wins";
        }
    }
    function distributeBetAmount() public payable{
        if (result == 1) {
            payable(owner).transfer(2000);
            BalanceOf[A] += 2000;
        }
        else if (result == 2) {
            payable(owner).transfer(2000);
            BalanceOf[B] += 2000;
        }
        else {
            payable(owner).transfer(2000);
        }
    }
    function gameresult() public view returns(uint, uint, uint) {
        return (BalanceOf[A], BalanceOf[B], address(this).balance);
    }
}

Overwriting src/Rsp.sol


In [2]:
!solc-windows.exe --optimize --combined-json abi,bin src/Rsp.sol > src/Rsp.json

In [3]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

var _abiBinJson = require('./Rsp.json');
var contractName = Object.keys(_abiBinJson.contracts);

_abi = _abiBinJson.contracts[contractName].abi;
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();

    await new web3.eth.Contract(_abi).deploy({data: "0x" + _bin}).estimateGas(function(err, gas) {
        if(!err) console.log("gas: " + gas);
    })

    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas:2000000}, function(err, transactionHash) {
            if(!err) console.log("hash: " + transactionHash);
        });
    console.log("The contract deployed to: " + deployed.options.address);
}
deploy()

Overwriting src/RspDeploy.js


In [4]:
!node src/RspDeploy.js

gas: 387899
hash: 0x6c4275088d2b1aa1e607923b0f6158eb577803ccf96367bd0cee2eb2abf7c79b
The contract deployed to: 0x4583932c8a147AD6Cf4D5Fc03471eBb6F421087b


In [5]:
%%writefile src/RspUse.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

var _abiBinJson = require('./Rsp.json');
var contractName = Object.keys(_abiBinJson.contracts);

abi = _abiBinJson.contracts[contractName].abi;
addr = "0x4583932c8a147AD6Cf4D5Fc03471eBb6F421087b";

rsp = new web3.eth.Contract(abi, addr);
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    await rsp.methods.setA().send({from: accounts[1], value: 1000, gas:200000});
    await rsp.methods.setB(2).send({from: accounts[2], value: 1000, gas:200000});
    await rsp.methods.play().send({from: accounts[0], gas:200000});
    await rsp.methods.gameresult().call().then(console.log);
    await rsp.methods.getMatchResult().call().then(console.log);
    await rsp.methods.distributeBetAmount().send({from: accounts[0], gas:200000});
    await rsp.methods.gameresult().call().then(console.log);
}
doIt();

Overwriting src/RspUse.js


In [6]:
!node src/RspUse.js

Result {
  '0': '999999346280292998000',
  '1': '999999365407073598000',
  '2': '2000'
}
B wins
Result {
  '0': '999999346280292998000',
  '1': '999999365407073600000',
  '2': '0'
}
